In [1]:
import wandb
import re
from pprint import pprint
import pandas as pd

wandb.login()
api = wandb.Api()
experiments = api.runs("rom42pla_team/noisy_eeg")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
table_rows = []
for experiment in experiments:
    date, hour, dataset, validation, signal_type, model = experiment.name.split(
        "_")
    df_experiment = experiment.history()
    runs = {col.split("-")[0] for col in df_experiment.columns if re.fullmatch(r"run_[0-9]+-.*", col)}
    avg_rows = []
    for run in runs:
        # prints all columns with the metrics of the run
        metrics = {col for col in df_experiment.columns if re.fullmatch(f"{run}-.*/.*", col)}
        # get the results for the run
        metrics_val = {col for col in df_experiment.columns if re.fullmatch(f"{run}-.*/val", col)}
        # gets the subset of dataframe for the run
        df_run = df_experiment[sorted(metrics_val)].dropna().reset_index(drop=True)
        df_run = df_run.rename(columns={col: col.split("-")[-1].split("/")[0] for col in df_run})
        # gets the best run based on the cls loss
        best_row_i = df_run["cls_loss"].idxmin()
        best_row = df_run.iloc[best_row_i]
        avg_rows.append(best_row)
    df_rows = pd.DataFrame(avg_rows)
    rows_mean, rows_std = df_rows.mean(), df_rows.std()
    for df in [rows_mean, rows_std]:
        df["dataset"] = dataset
        df["validation"] = validation
        df["signal_type"] = "<100Hz" if signal_type == "eeg" else ">100Hz"
        df["model"] = model
    rows = pd.DataFrame({
        key: f"{mean * 100:.3f} ± {std*100:.3f}" if isinstance(mean, float) else mean
        for key, mean, std in zip(rows_mean.index, rows_mean, rows_std)
    }, index=[0]).iloc[0]
    table_rows.append(rows)
# merge each run's series
table = pd.DataFrame(table_rows, dtype="object")
# reorder the columns
first_cols = ["dataset", "signal_type", "validation", "model"]
table = table.sort_values(first_cols)
table = table[first_cols +
              [col for col in table.columns if col not in first_cols]]

In [7]:
print(table.to_latex(na_rep="-", index=False))

\begin{tabular}{llllllllll}
\toprule
dataset & signal_type & validation & model & cls_acc & cls_f1 & cls_loss & ids_acc & ids_f1 & ids_loss \\
\midrule
deap & <100Hz & kfold & dino & 99.881 ± 0.043 & 99.898 ± 0.036 & 0.433 ± 0.208 & 99.971 ± 0.045 & 99.971 ± 0.045 & 0.139 ± 0.181 \\
deap & <100Hz & kfold & linear & 66.380 ± 0.658 & 74.336 ± 0.845 & 61.219 ± 0.445 & 99.620 ± 0.108 & 99.620 ± 0.108 & 3.983 ± 0.606 \\
deap & <100Hz & kfold & linear & 66.516 ± 0.656 & 74.173 ± 0.640 & 61.245 ± 0.438 & 99.598 ± 0.103 & 99.598 ± 0.103 & 4.217 ± 0.790 \\
deap & <100Hz & kfold & mlp & 83.628 ± 0.707 & 86.194 ± 0.855 & 37.063 ± 1.302 & 99.783 ± 0.108 & 99.783 ± 0.108 & 1.061 ± 0.605 \\
deap & <100Hz & loso & dino & 54.932 ± 6.350 & 59.780 ± 12.712 & 119.795 ± 60.725 & - & - & - \\
deap & <100Hz & loso & linear & 59.168 ± 5.179 & 67.237 ± 9.756 & 67.735 ± 3.567 & - & - & - \\
deap & <100Hz & loso & mlp & 57.272 ± 6.619 & 64.054 ± 10.290 & 71.434 ± 5.749 & - & - & - \\
deap & >100Hz & kfold & din